<a href="https://colab.research.google.com/github/vfrantc/weather_experiments/blob/main/fixed_range_train_decomp_residual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Jul 17 16:16:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from glob import glob
import shutil

In [ ]:
!pip install kornia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 517 kB 14.1 MB/s 


In [ ]:
import kornia as K

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!git clone https://github.com/vfrantc/deweather.git

Cloning into 'deweather'...
remote: Enumerating objects: 469, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 469 (delta 6), reused 2 (delta 2), pack-reused 456
Receiving objects: 100% (469/469), 14.02 MiB | 18.82 MiB/s, done.
Resolving deltas: 100% (265/265), done.


In [ ]:
%cd deweather/TransWeather-dehaze/

/content/deweather/TransWeather-dehaze


In [ ]:
!cp /content/drive/MyDrive/haze/data/SOTS.zip .
!cp /content/drive/MyDrive/deweather2/reside.zip .
!unzip SOTS.zip
!unzip reside.zip 
!mkdir -p data/train/
!mkdir -p data/test/outdoor/
!cp -r hazy data/train/input 
!cp -r SOTS/outdoor/hazy data/test/outdoor/input

Streaming output truncated to the last 5000 lines.
  inflating: hazy/1822_0.8_0.08.jpg  
  inflating: hazy/1823_0.95_0.1.jpg  
  inflating: hazy/1828_0.9_0.08.jpg  
  inflating: hazy/1832_0.95_0.06.jpg  
  inflating: hazy/1875_0.8_0.06.jpg  
  inflating: hazy/1889_1_0.2.jpg     
  inflating: hazy/1911_0.9_0.06.jpg  
  inflating: hazy/1915_0.95_0.04.jpg  
  inflating: hazy/1971_0.85_0.2.jpg  
  inflating: hazy/1976_0.9_0.04.jpg  
  inflating: hazy/1977_0.9_0.2.jpg   
  inflating: hazy/1977_1_0.08.jpg    
  inflating: hazy/1979_0.85_0.12.jpg  
  inflating: hazy/1982_0.95_0.12.jpg  
  inflating: hazy/2000_0.85_0.12.jpg  
  inflating: hazy/2018_0.85_0.2.jpg  
  inflating: hazy/2020_0.9_0.1.jpg   
  inflating: hazy/2020_1_0.2.jpg     
  inflating: hazy/2026_0.9_0.04.jpg  
  inflating: hazy/2033_0.9_0.08.jpg  
  inflating: hazy/2033_0.95_0.08.jpg  
  inflating: hazy/2043_0.95_0.04.jpg  
  inflating: hazy/2060_0.8_0.04.jpg  
  inflating: hazy/2070_0.9_0.16.jpg  
  inflating: hazy/2077_0.8_0.0

In [ ]:
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

import cv2
import numpy as np
from glob import glob
from tqdm.notebook import tqdm

class DecomNet(nn.Module):
    def __init__(self, channel=64, kernel_size=5):
        super(DecomNet, self).__init__()
        # Shallow feature extraction
        self.net1_conv0 = nn.Conv2d(15, channel, kernel_size * 3,  padding=7, padding_mode='replicate')
        # Activated layers!
        self.net1_convs = nn.Sequential(nn.Conv2d(channel, channel, kernel_size, padding=2, padding_mode='replicate'),
                                        nn.ReLU(),
                                        nn.Conv2d(channel, channel, kernel_size, padding=2, padding_mode='replicate'),
                                        nn.ReLU(),
                                        nn.Conv2d(channel, channel, kernel_size, padding=2, padding_mode='replicate'),
                                        nn.ReLU(),
                                        nn.Conv2d(channel, channel, kernel_size, padding=2, padding_mode='replicate'),
                                        nn.ReLU(),
                                        nn.Conv2d(channel, channel, kernel_size, padding=2, padding_mode='replicate'),
                                        nn.ReLU())
        # Final recon layer
        self.net1_recon = nn.Conv2d(channel, 6, kernel_size, padding=2, padding_mode='replicate')

    def forward(self, input_im):
        #input_img = torch.cat((input_im, input_im), dim=1)

        grads = K.filters.spatial_gradient(input_im, order=1)  # BxCx2xHxW
        grads1_x = grads[:, :, 0]
        grads1_y = grads[:, :, 1]
        grads = K.filters.spatial_gradient(input_im, order=2)  # BxCx2xHxW
        grads2_x = grads[:, :, 0]
        grads2_y = grads[:, :, 1]

        input = torch.cat((input_im, grads1_x, grads1_y, grads2_x, grads2_y), dim=1)

        feats0   = self.net1_conv0(input)
        featss   = self.net1_convs(feats0)
        outs     = self.net1_recon(featss)
        R        = torch.sigmoid(outs[:, 0:3, :, :])
        L        = torch.sigmoid(outs[:, 3:6, :, :])
        return R, L

def get_decom(trainable=True):
  net = DecomNet().cuda()
  ckpt_dict  = torch.load('decomp.tar') # , map_location=torch.device('cpu')
  net.load_state_dict(ckpt_dict)
  for p in net.parameters():
      p.requires_grad = trainable
  return net

def decom_image(image, net):
  test_low_img   = 2 * cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0 - 1
  test_low_img   = np.transpose(test_low_img, (2, 0, 1))
  input_low_test = np.expand_dims(test_low_img, axis=0)
  input_low_test = Variable(torch.FloatTensor(torch.from_numpy(input_low_test))).cuda()
  R_low, I_low   = net(input_low_test)
  R_low = np.clip(np.transpose(R_low.cpu().detach().numpy().squeeze(), (1, 2, 0)), 0, 1)
  I_low = np.clip(np.transpose(I_low.cpu().detach().numpy().squeeze(), (1, 2, 0)), 0, 1)
  return R_low, I_low

def process_mask(mask, net):
  for fname in tqdm(list(glob(mask+'*.png')) + list(glob(mask+'*.jpg'))):
    image = cv2.imread(fname)
    reflectance, illumination = decom_image(image, net)
    reflectance = (reflectance*255).astype(np.uint8)
    cv2.imwrite(fname, reflectance)

In [ ]:
!cp /content/drive/MyDrive/deweather2/decomp.tar .
net = get_decom()

In [ ]:
process_mask('data/train/input/', net)
process_mask('data/train/gt', net)
process_mask('data/test/outdoor/input/', net)
process_mask('data/test/outdoor/gt/',net)

  0%|          | 0/72135 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [ ]:
def process_dir(input_dir='data/train/input/', 
                gt_in='clear/', 
                gt_out='data/train/gt', 
                out_fname='data/train/reside.txt',
                preffix='input/'):
  if not os.path.exists(gt_out):
    os.makedirs(gt_out)
  
  with open(out_fname, 'w') as f:
    for fname in tqdm(glob(os.path.join(input_dir, '*.jpg'))):
      main_name = os.path.basename(fname)
      gt_name = main_name[:4] + '.jpg'
      try:
        shutil.copyfile(os.path.join(gt_in, gt_name), os.path.join(gt_out, main_name))
      except FileNotFoundError:
        img = cv2.imread(os.path.join(gt_in, gt_name).replace('.jpg', '.png'))
        cv2.imwrite(os.path.join(gt_out, main_name), img)
      f.write(preffix + main_name + '\n')

In [ ]:
print('process train')
process_dir(input_dir='data/train/input/', gt_in='clear/', gt_out='data/train/gt', out_fname='data/train/reside.txt', preffix='input/')
print('process outdoor')
process_dir(input_dir='data/test/outdoor/input/', gt_in='SOTS/outdoor/gt/', gt_out='data/test/outdoor/gt/', out_fname='data/test/outdoor.txt', preffix='outdoor/input/')

process train


  0%|          | 0/72135 [00:00<?, ?it/s]

process outdoor


  0%|          | 0/500 [00:00<?, ?it/s]

# Train the same network for dehazing (for 10 epochs and see how it works)

In [ ]:
!mkdir ./training_log/

In [ ]:
%cd /content/deweather/TransWeather-dehaze


/content/deweather/TransWeather-dehaze


In [ ]:
%cd /content/deweather/TransWeather-dehaze

/content/deweather/TransWeather-dehaze


In [ ]:
!python3 train.py  -train_batch_size 16 -exp_name Transweather -epoch_start 0 -num_epochs 20

In [ ]:
!zip -r despekle_20ep_sigmoid.zip ./Transweather/
!cp despekle_20ep_sigmoid.zip /content/drive/MyDrive/deweather

In [ ]:
!python test_dehaze.py -exp_name=./Transweather 

In [ ]:
!zip -r spekle_out7.zip results
!cp spekle_out7.zip /content/drive/MyDrive/deweather2